# Проект по SQL. Анализ базы данных сервиса для чтения книг по подписке

### Описание проекта

Проанализировать базу данных сервиса для чтения книг по подписке. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

**Цель исследования:** составить ценностное предложение для клиента исходя из данных о книгах, авторах, оценок критиков, обзоров. По результату исследования можно ответить на вопрос: какие книги с наибольшей вероятностью принесут новой компании прибыль и популярность. 

## Загрузка данных. Первый взгляд.

Загрузим библиотеки и посмотрим на структуру предоставленных данных.

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

In [ ]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
 'pwd': 'Sdf4$2;d-d30pp', # пароль
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432, # порт подключения
 'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

* Таблица **books** с информацией о книгах

In [ ]:
query = '''
        SELECT *
        FROM books
        LIMIT 5;
        '''
pd.io.sql.read_sql(query, con = engine) 

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


* Таблица **authors** с информацией об авторах книг

In [ ]:
query = '''
        SELECT *
        FROM authors
        LIMIT 5;
        '''
pd.io.sql.read_sql(query, con = engine) 

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


* Таблица **publishers** с информацией об авторах книг

In [ ]:
query = '''
        SELECT *
        FROM publishers
        LIMIT 5;
        '''
pd.io.sql.read_sql(query, con = engine) 

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


* Таблица **ratings** с информацией об авторах книг

In [ ]:
query = '''
        SELECT *
        FROM ratings
        LIMIT 5;
        '''
pd.io.sql.read_sql(query, con = engine) 

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


* Таблица **reviews** с информацией об авторах книг

In [ ]:
query = '''
        SELECT *
        FROM reviews
        LIMIT 5;
        '''
pd.io.sql.read_sql(query, con = engine) 

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


### Посчитаем, сколько книг вышло после 1 января 2000 года;

Напишем функцию для упрощения вывода результата запроса.

In [ ]:
def reqst(q):
    query = q
    return pd.io.sql.read_sql(query, con = engine) 

In [ ]:
reqst(
'''
SELECT COUNT(book_id)
FROM books
WHERE publication_date > '2000-01-01';
'''
)

,count
0,819


### Вывод

После 1 января 2000 года вышло **819 книг.** Новые книги появляются. Их необходимо продвигать и предлагать пользователям.

<div class="alert alert-success">
    Отлично
</div>

### Для каждой книги посчитаем количество обзоров и среднюю оценку

In [ ]:
reqst(
'''
WITH
    i AS (
SELECT book_id,
        COUNT(review_id) AS amount_review        
FROM reviews
GROUP BY book_id
),
    a AS (
SELECT book_id,
        AVG(rating) AS avg_rating
FROM ratings
GROUP BY book_id
)

SELECT title,
        i.amount_review,
        a.avg_rating
FROM books AS b
LEFT JOIN a ON b.book_id = a.book_id
LEFT JOIN i ON b.book_id = i.book_id
ORDER BY avg_rating DESC;

'''
)

,title,amount_review,avg_rating
0,A Dirty Job (Grim Reaper #1),4.0,5.00
1,Tai-Pan (Asian Saga #2),2.0,5.00
2,School's Out—Forever (Maximum Ride #2),3.0,5.00
3,Piercing the Darkness (Darkness #2),2.0,5.00
4,The War of Art: Break Through the Blocks & Win...,2.0,5.00
...,...,...,...
995,The World Is Flat: A Brief History of the Twen...,3.0,2.25
996,His Excellency: George Washington,2.0,2.00
997,Junky,2.0,2.00
998,Drowning Ruth,3.0,2.00


### Вывод

Не у всех книг есть обзоры. Данные отсортированы по убыванию рейтинга. Этот список поможет понять, как ранжировать книги в библиотеке и рекомендательном сервисе. Хороший пользовательский опыт важен для удержания клиента. 

<div class="alert alert-success">
    Отлично
</div>

### Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так исключим из анализа брошюры

In [ ]:
reqst(
'''
WITH
    i AS (
SELECT publisher_id,
        COUNT(book_id) AS amount_books
FROM books
WHERE num_pages > 50
GROUP BY publisher_id
        )
        
SELECT publisher,
        amount_books
FROM publishers AS p
INNER JOIN i ON p.publisher_id = i.publisher_id
ORDER BY i.amount_books DESC
LIMIT 5;
'''
)

,publisher,amount_books
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Bantam,19


### Вывод

Penguin Books лидирует по количеству, выпущенных книг свыше 50 страниц.

<div class="alert alert-success">
    Отлично
</div>

### Определим автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

In [ ]:
reqst(
'''
WITH
    i AS (
SELECT book_id,
       COUNT(rating) AS amount_rating
FROM ratings
GROUP BY book_id
HAVING COUNT(rating) > 50
        ),
    p AS (
SELECT rs.book_id,
       rs.rating 
FROM ratings AS rs
INNER JOIN i ON rs.book_id = i.book_id
        ),
    b AS (
SELECT bs.author_id,
       AVG(p.rating) AS avg_rating 
FROM books AS bs
INNER JOIN p ON bs.book_id = p.book_id
GROUP BY bs.author_id
        )
SELECT at.author,
       b.avg_rating
FROM authors AS at
INNER JOIN b ON at.author_id = b.author_id
ORDER BY b.avg_rating DESC;
'''
)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.287097
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.246914
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Lois Lowry,3.750000


### Вывод

Получили список с авторами у которых есть книги с 50 и более оценками. Ранжировали их по среднему рейтингу. Дж. Роулинг лидирует. Гарри Поттер рулит) А вот второе место мне неизвестный, но нагуглил, что "книжный вор" его работа. Замыкает тройку Толкиен и "Властелин Колец". В целом ожидаемый результат, кроме 2-го места. 

### Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [ ]:
reqst(
'''
WITH
    i AS (
SELECT username,
       COUNT(rating)
FROM ratings
GROUP BY username
HAVING COUNT(rating) > 50
    ),
    a AS (
SELECT COUNT(review_id) AS amount_reviews
FROM reviews AS rs
INNER JOIN i ON i.username = rs.username
GROUP BY rs.username
            )
SELECT AVG(amount_reviews) AS average_amount_reviews
FROM a
'''
)

,average_amount_reviews
0,24.333333


### Вывод

В среднем **24** обзора делают пользователи, оценившие более 50 книг. Это пользователи проактивные. Уникальный контент от пользователей хороший буст в продажах. Хорошо знать таких пользователей для того, чтобы предложить им релевантные книжные новинки, которые они с большей вероятностью оценят и дадут подробную обратную связь в виде обзора. Если это сработает, то будет хорошее промо для книги.  